<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/demo_basic_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic NLP exercises

* During these exercises, you will learn basic Python skills required in NLP, for example
  * Reading and processing language data
  * Segmenting text
  * Calculating word frequencies and idf weights

* Exercises are based on tweets downloaded using Twitter API. Both Finnish and English tweets are available, you are free to choose which language you want to work with.


> Finnish: http://dl.turkunlp.org/intro-to-nlp/finnish-tweets-sample.jsonl.gz

> English: http://dl.turkunlp.org/intro-to-nlp/english-tweets-sample.jsonl.gz


* Both files include 10,000 tweets. If processing the whole file takes too much time, you can also read just a subset of the data, for example only 1,000 tweets.


## 1) Read tweets in Python

* Download the file, and read the data in Python
* **The outcome of this exercise** should be a list of tweets, where each tweet is a dictionary including different (key, value) pairs

In [ ]:
# write your code here and run the cell to see your output
!wget -nc http://dl.turkunlp.org/intro-to-nlp/english-tweets-sample.jsonl.gz

import gzip
import json

with gzip.open("english-tweets-sample.jsonl.gz", "rt", encoding="utf-8") as f:
    lines = f.readlines()[:1000]

print(type(lines[0]))
print(lines[0])

tweets = []
for line in lines:
    data = json.loads(line) #.load()
    tweets.append(data)

print(len(tweets))

## 2) Extract texts from the tweet jsons

* During these exercises we need only the actual tweet text. Inspect the dictionary and extract the actual text field for each tweet.
* When carefully inspecting the dictionary keys and values, you may see the old Twitter character limit causing unexpect behavior for text. In these cases, are you able to extract the full text?
* **The outcome of this exercise** should be a list of tweets, where each tweet is a string.

In [ ]:
# going to back to the dictionaries to look for truncated ones
for tweet in tweets:
    # note that endswith() checks for one character called ellipsis, not for three dots
    if tweet["text"].endswith("…"):

        # note that the textfield ends with an ellipsis
        print("Text field:", tweet['text'])

        # you can also see that a display_url entity is shortened with an ellipsis
        print("\nDictionary:", tweet)

        # the first truncated tweet seems to be a retweet, and the original tweet is under retweeted_status
        # however, under the retweet_status the text can still be truncated
        print("\nretweeted_status/text:", tweet["retweeted_status"]["text"])

        # under the retweet_status is extended_tweet/full_text which will reveal the full original tweet
        if "extended_tweet" in tweet["retweeted_status"]:
            print("\nretweeted_status/extended_tweet/full_text:", tweet["retweeted_status"]["extended_tweet"]["full_text"])

        # if retweet then look for original
        # if original has extended_tweet field use it
        # else use normal text field

        # stop after finding first truncated tweet
        break


In [ ]:
def get_full_text(tweet):

    # replaces the retweet with the original tweet if retweet_status key exists
    if "retweeted_status" in tweet:
        tweet = tweet["retweeted_status"]
    
    # creates a text variable from the full_text field from extended_tweet key if it exists
    # else just creates text variable from the original text
    if "extended_tweet" in tweet:
        text = tweet["extended_tweet"]["full_text"]
    else:
        text = tweet["text"]

    # replaces line breaks with whitespaces
    # i.e. print one tweet per line
    text = text.replace("\n", " ")

    return text

tweets_text = []
for tweet in tweets:
    tweets_text.append(get_full_text(tweet))

print("Number of tweets: ", len(tweets_text))
print("Type: ", type(tweets_text[0]))
print("First tweet: ", tweets_text[0])


# print the first x tweets
x = 10
for i, t in enumerate(tweets_text[:x]):
    print(i, t)

## 3) Segment tweets

* Segment tweets using the UDPipe machine learned model, remember to select the correct language.

> English model: https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/en.segmenter.udpipe

> Finnish model: https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/fi.segmenter.udpipe

* Note that the segmentation model was not trained on tweets, so it may have difficulties in some cases. Inspect the output to get an idea how well it performs on tweets.
* Note: In case of the notebook cell dies while trying to load/run the model, the most typical reason is wrong file path or name, or incorrectly downloaded model.
* **The output of this excercise** should be a list of segmented tweets, where each tweet is a string.

In [ ]:
!pip install ufal.udpipe
!wget -nc https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/en.segmenter.udpipe

In [ ]:
import re

# replace . , ! ? characters with whitespace+character(s), '+' means one or more
def heuristic_tokenizer(text):
    tokenized = re.sub(r'([.,!?]+)', r' \1', text)
    tokenized = re.sub(r"(n't)", r" \1", tokenized)
    return tokenized

heuristic_segmented = [heuristic_tokenizer(tweet) for tweet in tweets_text]

import ufal.udpipe as udpipe

# create segmenter model
model = udpipe.Model.load("en.segmenter.udpipe")
pipeline = udpipe.Pipeline(model,"tokenize","none","none","horizontal")

udpipe_segmented = [pipeline.process(tweet) for tweet in tweets_text]

for i in range(5):
    print("Heuristic: ", heuristic_segmented[i])
    print("Udpipe: ", udpipe_segmented[i])

## 4) Calculate word frequencies

* Calculate a word frequency list (how many times each word appears) based on the tweets. Which are the most common words appearing in the data?
* Calculate the size of your vocabulary (how many unique words there are).
* **The output of this excercise** should be a sorted list of X most common words and their frequencies, and the number of unique words in the data.

## 5) Calculate idf weights

* Calculate idf weight for each word appearing in the data (one tweet = one document), and print top 20 words with lowest and highest idf values.
* Can you think of a reason why someone could claim that tf does not have a high impact when processing tweets?
* **The output of this excercise** should be a list of words sorted by their idf weights.


## 6) Duplicates or near duplicates

* Check whether we have duplicate tweets (in terms of text field only) in our dataset. Duplicate tweet means here that the exactly same tweet text appears more than once in our dataset.
* Note: It makes sense to check the duplicates using original tweet texts as the texts were before segmentation. I would also recommend using the full 10,000 dataset here in order to get higher chance of seeing duplicates (this does not require heavy computing).
* Try to check whether tweets have additional near-duplicates. Near duplicate means here that tweet text is almost the same in two or more tweets. Ponder what kind of near duplicates there could be and how to find those. Start by considering for example different normalization techniques. Implement some of the techniques you considered.
* **The outcome of this exercise** should be a number of unique tweets in our dataset (with possibly counting also which are the most common duplicates) as well as the number of unique tweets after removing also near duplicates.